In [43]:
import yfinance as yf
import os, contextlib
import pandas as pd

In [44]:
offset = 0
limit = None
period = '10y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [48]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N'].iloc[8805:]
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 2618


In [47]:
data_clean[data_clean['Symbol'] == 'ROCLW']

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
8805,Y,ROCLW,Roth CH Acquisition V Co. - Warrant,Q,G,N,100.0,N,N,NaN,ROCLW,N


In [49]:
%%time

limit = limit if limit else len(symbols)
end = max(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            print(i)
            s = symbols[i]
            print(s)
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 2456
CPU times: user 3min 15s, sys: 4.1 s, total: 3min 19s
Wall time: 12min 34s


In [10]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

In [11]:
!mkdir stocks
!mkdir etfs

In [12]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [ ]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")